In [ ]:
pip install --upgrade pandas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import netCDF4 as nc
from netCDF4 import Dataset
import csv
from geojson import Point, Feature, FeatureCollection

/Users/caydenwilson/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
no2 = xr.open_dataset("Land_and_Ocean_LatLong1.nc")
no2

<xarray.Dataset>
Dimensions:      (longitude: 360, latitude: 180, time: 2083, month_number: 12)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * time         (time) float64 1.85e+03 1.85e+03 ... 2.023e+03 2.024e+03
Dimensions without coordinates: month_number
Data variables:
    land_mask    (latitude, longitude) float64 ...
    temperature  (time, latitude, longitude) float32 ...
    climatology  (month_number, latitude, longitude) float32 ...
Attributes:
    Conventions:           Berkeley Earth Internal Convention (based on CF-1.5)
    title:                 Native Format Berkeley Earth Surface Temperature A...
    history:               26-Aug-2023 08:25:17
    institution:           Berkeley Earth Surface Temperature Project
    land_source_history:   03-Aug-2023 08:11:01
    ocean_source_history:  26-Aug-2023 05:48:52
    comment:               This file contains Berkeley Earth surface temperat...

In [113]:
no4 = no2.sel(time=slice(2023,2024))

In [114]:
t_df = no4['temperature'].to_dataframe(name = None)

In [115]:
t_df

temperature
time        latitude longitude             
2023.041667 -89.5    -179.5       -2.791210
                     -178.5       -2.788814
                     -177.5       -2.786446
                     -176.5       -2.784106
                     -175.5       -2.781793
...                                     ...
2023.541667  89.5     175.5        2.528067
                      176.5        2.529688
                      177.5        2.531455
                      178.5        2.533216
                      179.5        2.534972

[453600 rows x 1 columns]

In [116]:
t_df = t_df.dropna()

In [117]:
t_df.to_csv('t_df.csv')

In [118]:
df1 = pd.read_csv('t_df.csv')
df1['time'] = df1['time'].astype('int')
df1.to_csv('output2.csv')

In [119]:
def csv_to_geojson(csv_file):
    features = []
    with open(csv_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            latitude = float(row['latitude'])
            longitude = float(row['longitude'])
            time = row['time']
            temperature = float(row['temperature'])

            # Create a GeoJSON point feature
            point = Point((longitude, latitude))
            properties = {
                'time': time,
                'temperature': temperature
                # Add other properties as needed
            }
            features.append(Feature(geometry=point, properties=properties))

    # Create a FeatureCollection
    feature_collection = FeatureCollection(features)
    return feature_collection

# Replace 'input.csv' with your CSV file name
input_csv_file = 'output2.csv'
output_geojson_file = 'output2023.geojson'

geojson_data = csv_to_geojson(input_csv_file)

# Write GeoJSON data to a file
with open(output_geojson_file, 'w') as output_file:
    output_file.write(str(geojson_data))


In [26]:
emissions = pd.read_csv('GCB2022v27_MtCO2_flat.csv')

In [27]:
output = emissions[(emissions['Year'] == 2021)]

In [28]:
output

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
271,Afghanistan,AFG,2021,11.874211,4.203536,7.419570,0.238847,0.012259,0.000000,NaN,0.296119
543,Albania,ALB,2021,4.619109,0.174413,3.246273,0.150245,1.048179,0.000000,NaN,1.618066
815,Algeria,DZA,2021,176.269070,1.095536,56.020598,92.791955,11.028039,15.332942,NaN,3.989977
1087,Andorra,AND,2021,0.452888,0.000000,0.452888,0.000000,0.000000,0.000000,NaN,5.730292
1359,Angola,AGO,2021,21.362717,0.000000,13.901716,2.951298,1.123730,3.385974,NaN,0.619141
...,...,...,...,...,...,...,...,...,...,...,...
62015,Yemen,YEM,2021,12.476597,0.245909,9.592510,0.206611,0.663596,1.767971,NaN,0.378289
62287,Zambia,ZMB,2021,7.676180,2.463629,4.251568,0.000000,0.960983,0.000000,NaN,0.394194
62559,Zimbabwe,ZWE,2021,11.296114,7.225620,3.500879,0.000000,0.569615,0.000000,NaN,0.706293
62831,International Transport,XIT,2021,1021.749204,NaN,1021.749204,NaN,NaN,NaN,NaN,NaN


In [29]:
output[['Country', 'Total']].to_csv('emissions.csv', index=False)

PLANNING:
1. Freeze data to one year and reshape 2 dimensional
2. Get mapbox example working for one year

Potentially implement drop down for year or sldier
